In [1]:
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Bidirectional, LSTM, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

# Define the input shape
input_data = Input(shape=(82, 64, 32, 1))  # frame, height, width, channels

# First convolutional block
conv1 = TimeDistributed(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))(input_data)
bn1 = TimeDistributed(BatchNormalization())(conv1)
conv1 = TimeDistributed(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))(bn1)
bn1 = TimeDistributed(BatchNormalization())(conv1)
pool1 = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(bn1)

# Second convolutional block
conv2 = TimeDistributed(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))(pool1)
bn2 = TimeDistributed(BatchNormalization())(conv2)
conv2 = TimeDistributed(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))(bn2)
bn2 = TimeDistributed(BatchNormalization())(conv2)
pool2 = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(bn2)

# Third convolutional block
conv3 = TimeDistributed(Conv2D(filters=256, kernel_size=(3, 3), activation='relu'))(pool2)
bn3 = TimeDistributed(BatchNormalization())(conv3)
conv3 = TimeDistributed(Conv2D(filters=256, kernel_size=(3, 3), activation='relu'))(bn3)
bn3 = TimeDistributed(BatchNormalization())(conv3)
pool3 = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(bn3)

# Flatten the output of the last pooling layer
flatten = TimeDistributed(Flatten())(pool3)

# Fully connected layers
dense = TimeDistributed(Dense(24))(flatten)

# Bidirectional LSTM layers
blstm = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1))(dense)
blstm = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1))(blstm)
y_pred = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1))(blstm)

# Output dense layer
dense_output = TimeDistributed(Dense(90 + 1, name="dense"))(y_pred)

# Define the model
model = Model(inputs=input_data, outputs=dense_output)

# Define the CTC loss function
def ctc_loss(y_true, y_pred):
    input_length = np.ones((y_pred.shape[0], 1)) * y_pred.shape[1]
    label_length = np.sum(y_true != 0, axis=1)
    return K.ctc_batch_cost(y_true, y_pred, input_length, label_length)

# Compile the model with CTC loss
model.compile(optimizer=Adam(learning_rate=0.0001), loss=ctc_loss)

# Display the model summary
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 82, 64, 32, 1)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 82, 62, 30, 64) │           640 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 82, 62, 30, 64) │           256 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 82, 60, 28, 64) │        36,928 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 82, 60, 28, 64) │           256 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 82, 30, 14, 64) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 82, 28, 12,     │        73,856 │
│ (TimeDistributed)               │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_6              │ (None, 82, 28, 12,     │           512 │
│ (TimeDistributed)               │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_7              │ (None, 82, 26, 10,     │       147,584 │
│ (TimeDistributed)               │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_8              │ (None, 82, 26, 10,     │           512 │
│ (TimeDistributed)               │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_9              │ (None, 82, 13, 5, 128) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_10             │ (None, 82, 11, 3, 256) │       295,168 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_11             │ (None, 82, 11, 3, 256) │         1,024 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_12             │ (None, 82, 9, 1, 256)  │       590,080 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_13             │ (None, 82, 9, 1, 256)  │         1,024 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_14             │ (None, 82, 4, 0, 256)  │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_15             │ (None, 82, 0)          │             0 │
│ (TimeDistributed)               │                        │             

 Total params: 2,116,403 (8.07 MB)

 Trainable params: 2,114,611 (8.07 MB)

 Non-trainable params: 1,792 (7.00 KB)

In [ ]:
model.predict

In [1]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Dropout, Reshape, Dense, Bidirectional, LSTM, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

def build_image_text_recogniser_model(img_w, img_h, num_classes, drop_out_rate=0.35):
    """
    Builds the image text recognizer model.
    """
    input_shape = (1, img_w, img_h) if K.image_data_format() == 'channels_first' else (img_w, img_h, 1)
    
    model_input = Input(shape=input_shape, name='img_input', dtype='float32')

    # Convolutional layers
    x = Conv2D(64, (3, 3), padding='same', name='conv1', kernel_initializer='he_normal')(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2), name='max1')(x)

    x = Conv2D(128, (3, 3), padding='same', name='conv2', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2), name='max2')(x)

    x = Conv2D(256, (3, 3), padding='same', name='conv3', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same', name='conv4', kernel_initializer='he_normal')(x)
    x = Dropout(drop_out_rate)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(1, 2), name='max3')(x)

    x = Conv2D(512, (3, 3), padding='same', name='conv5', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv6')(x)
    x = Dropout(drop_out_rate)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(1, 2), name='max4')(x)

    x = Conv2D(512, (2, 2), padding='same', kernel_initializer='he_normal', name='conv7')(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Calculate the reshape target shape
    output_shape = K.int_shape(x)
    reshaped_shape = (output_shape[1] * output_shape[2], output_shape[3])

    # CNN to RNN
    x = Reshape(target_shape=reshaped_shape, name='reshape')(x)
    x = Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1')(x)

    # RNN layers
    x = Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='he_normal'), merge_mode='sum')(x)
    x = Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='he_normal'), merge_mode='concat')(x)

    # Output layer
    x = Dense(num_classes, kernel_initializer='he_normal', name='dense2')(x)
    y_pred = Activation('softmax', name='softmax')(x)

    return Model(inputs=model_input, outputs=y_pred)


In [2]:
# Define parameters
img_w, img_h = 128, 64  # Example dimensions
num_classes = 80  # Example number of classes
max_length = 32  # Example max length of the sequence
drop_out_rate = 0.35

# Build the model
model = build_image_text_recogniser_model(img_w, img_h, num_classes, drop_out_rate)

# Compile the model for training
#train_model = compile_model_for_training(model, img_w, img_h, num_classes, max_length)

# Now you can compile the train_model with an optimizer and loss function
#train_model.compile(optimizer='adam', loss={'ctc': lambda y_true, y_pred: y_pred})


In [3]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ img_input (InputLayer)          │ (None, 128, 64, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 128, 64, 64)    │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 64, 64)    │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 128, 64, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max1 (MaxPooling2D)             │ (None, 64, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 64, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 64, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max2 (MaxPooling2D)             │ (None, 32, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3 (Conv2D)                  │ (None, 32, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 32, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv4 (Conv2D)                  │ (None, 32, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 32, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max3 (MaxPooling2D)             │ (None, 32, 8, 256)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv5 (Conv2D)                  │ (None, 32, 8, 512)     │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 32, 8, 512)     │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 32, 8, 512)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv6 (Conv2D)                  │ (None, 32, 8, 512)     │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32, 8, 512)     │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 7,339,664 (28.00 MB)

 Trainable params: 7,335,184 (27.98 MB)

 Non-trainable params: 4,480 (17.50 KB)

In [4]:
def compile_model_for_training(model, img_w, img_h, num_classes, max_length):
    """
    Compiles the model for training by adding the CTC loss function.
    """
    labels = Input(name='ground_truth_labels', shape=[max_length], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')

    y_pred = model.output

    def ctc_loss_function(args):
        y_pred, labels, input_length, label_length = args
        return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

    loss_out = Lambda(ctc_loss_function, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])
    train_model = Model(inputs=[model.input, labels, input_length, label_length], outputs=loss_out)
    
    return train_model

In [ ]:
trained_model = compile_model_for_training()